# Features per player

These features were chosen based on a mix of general good features for tennis match analysis and longterm experience as a tennis players myself

## Basic information
- Player_id
- Player name (first and last)
- age at match
- current points ATP
- dominant hand
- dob
- ioc
- height
- current rank

## Career information
- percentage matches won
- grand slams percentage won
- wimbledon percentage won
- years on tour
- average rank opponents beaten
- common opponents difference


## Per Match Statistics
- average aces
- average double faults
- average first serve percentage
- average first serve points won
- first serve consistency score (percentage * won)
- average second serve points won
- average break points won
- average break points saved
- average winners
- average unforced errors
- average forced errors
- effectiveness score (winners - (un- + forced errors))
- average total points/sets won
- average serve speed

## Tourney information
- average level of tourney -> tourney score
- tourney round
- is home country
- match type (number of winning sets)
- percentage matches played on surface xy (hard, clay, grass, carpet)
- suface speed score

First look into the data to get to know the data structure

In [1]:
import pandas as pd

# Load a test file
match = pd.read_csv('../data/atp_matches/atp_matches_2019.csv')

print(match.head())
match.dtypes

  tourney_id tourney_name surface  draw_size tourney_level  tourney_date  \
0  2019-M020     Brisbane    Hard         32             A      20181231   
1  2019-M020     Brisbane    Hard         32             A      20181231   
2  2019-M020     Brisbane    Hard         32             A      20181231   
3  2019-M020     Brisbane    Hard         32             A      20181231   
4  2019-M020     Brisbane    Hard         32             A      20181231   

   match_num  winner_id  winner_seed winner_entry  ... l_1stIn l_1stWon  \
0        300     105453          2.0          NaN  ...    54.0     34.0   
1        299     106421          4.0          NaN  ...    52.0     36.0   
2        298     105453          2.0          NaN  ...    27.0     15.0   
3        297     104542          NaN           PR  ...    60.0     38.0   
4        296     106421          4.0          NaN  ...    56.0     46.0   

   l_2ndWon l_SvGms  l_bpSaved  l_bpFaced  winner_rank winner_rank_points  \
0      20.0    

tourney_id             object
tourney_name           object
surface                object
draw_size               int64
tourney_level          object
tourney_date            int64
match_num               int64
winner_id               int64
winner_seed           float64
winner_entry           object
winner_name            object
winner_hand            object
winner_ht             float64
winner_ioc             object
winner_age            float64
loser_id                int64
loser_seed            float64
loser_entry            object
loser_name             object
loser_hand             object
loser_ht              float64
loser_ioc              object
loser_age             float64
score                  object
best_of                 int64
round                  object
minutes               float64
w_ace                 float64
w_df                  float64
w_svpt                float64
w_1stIn               float64
w_1stWon              float64
w_2ndWon              float64
w_SvGms   

In [44]:
players = pd.read_csv('../data/atp_matches/atp_players.csv')

print(players['hand'].unique())
players['height'] = players['height'].fillna(players['height'].median())
print(players.head())
print(players.dtypes)

['R' 'L' 'U' 'A' nan]
   player_id name_first name_last hand         dob  ioc  height wikidata_id
0     100001    Gardnar    Mulloy    R  19131122.0  USA   185.0      Q54544
1     100002     Pancho    Segura    R  19210620.0  ECU   168.0      Q54581
2     100003      Frank   Sedgman    R  19271002.0  AUS   180.0     Q962049
3     100004   Giuseppe     Merlo    R  19271011.0  ITA   183.0    Q1258752
4     100005    Richard  Gonzalez    R  19280509.0  USA   188.0      Q53554
player_id        int64
name_first      object
name_last       object
hand            object
dob            float64
ioc             object
height         float64
wikidata_id     object
dtype: object


/var/folders/fm/xhk7242x0d30x60k6sqjyxl80000gn/T/ipykernel_54904/3459264720.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  players = pd.read_csv('../data/atp_matches/atp_players.csv')


In [45]:
rankings = pd.read_csv('../data/atp_matches/atp_rankings_current.csv')
rankings = rankings.sort_values(by='rank')

print(rankings.head())
rankings.columns

       ranking_date  rank  player  points
0          20240101     1  104925   11245
62449      20240923     1  206173   11180
64575      20240930     1  206173   11010
12340      20240219     1  104925    9855
66704      20241014     1  206173   11920


Index(['ranking_date', 'rank', 'player', 'points'], dtype='object')

In [46]:
# Combine rankings with players
rankings = rankings.rename(columns={'player': 'player_id'})

ranked_players = pd.merge(rankings, players, on='player_id', how='left')
ranked_players = ranked_players.sort_values(by = ['ranking_date','rank'], ascending = [False, True])
print(ranked_players.head())

     ranking_date  rank  player_id  points name_first name_last hand  \
42       20241230     1     206173   11830     Jannik    Sinner    R   
49       20241230     2     100644    7915  Alexander    Zverev    R   
105      20241230     3     207989    7010     Carlos   Alcaraz    R   
134      20241230     4     126203    5100     Taylor     Fritz    R   
209      20241230     5     106421    5030     Daniil  Medvedev    R   

            dob  ioc  height wikidata_id  
42   20010816.0  ITA   191.0   Q54812588  
49   19970420.0  GER   198.0   Q13990552  
105  20030505.0  ESP   183.0   Q85518537  
134  19971028.0  USA   196.0   Q17660516  
209  19960211.0  RUS   198.0   Q21622022  


In [47]:
# get latest atp rankings
latest_rankings = ranked_players.drop_duplicates(subset=['rank'], keep='first')
print(latest_rankings)
latest_rankings.columns

       ranking_date  rank  player_id  points name_first    name_last hand  \
42         20241230     1     206173   11830     Jannik       Sinner    R   
49         20241230     2     100644    7915  Alexander       Zverev    R   
105        20241230     3     207989    7010     Carlos      Alcaraz    R   
134        20241230     4     126203    5100     Taylor        Fritz    R   
209        20241230     5     106421    5030     Daniil     Medvedev    R   
...             ...   ...        ...     ...        ...          ...  ...   
85419      20240108  1920     205831       1      Ammar    Alhogbani    U   
70075      20240101  1586     212263       2   Xing Dao         Chen    R   
82949      20240101  1853     211341       1      Tommy  Czaplinski     U   
85521      20240101  1923     208036       1      Maxim     Krapivin    U   
89758      20240101  2023     211722       1     Manish       Ganesh    U   

              dob  ioc  height wikidata_id  
42     20010816.0  ITA   191.0

Index(['ranking_date', 'rank', 'player_id', 'points', 'name_first',
       'name_last', 'hand', 'dob', 'ioc', 'height', 'wikidata_id'],
      dtype='object')

In [48]:
# getting to know point by point data
# single matches not point by point
match_data = pd.read_csv('../data/slam_pointbypoint/2024-usopen-matches.csv')
print(match_data.head())
print(match_data.columns)

           match_id  year    slam  match_num                player1  \
0  2024-usopen-1101  2024  usopen       1101          Jannik Sinner   
1  2024-usopen-1102  2024  usopen       1102        Eliot Spizzirri   
2  2024-usopen-1103  2024  usopen       1103        Mattia Bellucci   
3  2024-usopen-1104  2024  usopen       1104  Christopher O'Connell   
4  2024-usopen-1105  2024  usopen       1105            Arthur Fils   

              player2  status  winner  event_name  round  court_name  \
0  Mackenzie McDonald     NaN     NaN         NaN    NaN         NaN   
1      Alex Michelsen     NaN     NaN         NaN    NaN         NaN   
2       Stan Wawrinka     NaN     NaN         NaN    NaN         NaN   
3       Nicolas Jarry     NaN     NaN         NaN    NaN         NaN   
4        Learner Tien     NaN     NaN         NaN    NaN         NaN   

   court_id  player1id  player2id  nation1  nation2  
0       NaN        NaN        NaN      NaN      NaN  
1       NaN        NaN        Na

In [55]:
# points data
points_data = pd.read_csv('../data/slam_pointbypoint/2011-ausopen-points.csv')
print(points_data.columns)
points_data.dtypes

Index(['match_id', 'ElapsedTime', 'SetNo', 'P1GamesWon', 'P2GamesWon',
       'SetWinner', 'GameNo', 'GameWinner', 'PointNumber', 'PointWinner',
       'PointServer', 'Speed_KMH', 'Rally', 'P1Score', 'P2Score', 'P1Momentum',
       'P2Momentum', 'P1PointsWon', 'P2PointsWon', 'P1Ace', 'P2Ace',
       'P1Winner', 'P2Winner', 'P1DoubleFault', 'P2DoubleFault', 'P1UnfErr',
       'P2UnfErr', 'P1NetPoint', 'P2NetPoint', 'P1NetPointWon',
       'P2NetPointWon', 'P1BreakPoint', 'P2BreakPoint', 'P1BreakPointWon',
       'P2BreakPointWon', 'P1FirstSrvIn', 'P2FirstSrvIn', 'P1FirstSrvWon',
       'P2FirstSrvWon', 'P1SecondSrvIn', 'P2SecondSrvIn', 'P1SecondSrvWon',
       'P2SecondSrvWon', 'P1ForcedError', 'P2ForcedError', 'History',
       'Speed_MPH', 'P1BreakPointMissed', 'P2BreakPointMissed',
       'ServeIndicator', 'P1TurningPoint', 'P2TurningPoint'],
      dtype='object')


match_id               object
ElapsedTime            object
SetNo                 float64
P1GamesWon            float64
P2GamesWon            float64
SetWinner               int64
GameNo                float64
GameWinner              int64
PointNumber             int64
PointWinner           float64
PointServer           float64
Speed_KMH             float64
Rally                 float64
P1Score                object
P2Score                object
P1Momentum              int64
P2Momentum              int64
P1PointsWon           float64
P2PointsWon           float64
P1Ace                 float64
P2Ace                 float64
P1Winner              float64
P2Winner              float64
P1DoubleFault         float64
P2DoubleFault         float64
P1UnfErr              float64
P2UnfErr              float64
P1NetPoint              int64
P2NetPoint              int64
P1NetPointWon           int64
P2NetPointWon           int64
P1BreakPoint            int64
P2BreakPoint            int64
P1BreakPoi

In [50]:
# add match_id to atp_matches
match['match_id'] = match['tourney_id'] + '-' + match['match_num'].astype('str')
match

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,match_id
0,2019-M020,Brisbane,Hard,32,A,20181231,300,105453,2.0,NaN,...,34.0,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0,2019-M020-300
1,2019-M020,Brisbane,Hard,32,A,20181231,299,106421,4.0,NaN,...,36.0,7.0,10.0,10.0,13.0,16.0,1977.0,239.0,200.0,2019-M020-299
2,2019-M020,Brisbane,Hard,32,A,20181231,298,105453,2.0,NaN,...,15.0,6.0,8.0,1.0,5.0,9.0,3590.0,40.0,1050.0,2019-M020-298
3,2019-M020,Brisbane,Hard,32,A,20181231,297,104542,NaN,PR,...,38.0,9.0,11.0,4.0,6.0,239.0,200.0,31.0,1298.0,2019-M020-297
4,2019-M020,Brisbane,Hard,32,A,20181231,296,106421,4.0,NaN,...,46.0,19.0,15.0,2.0,4.0,16.0,1977.0,18.0,1855.0,2019-M020-296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2801,2019-9210,Laver Cup,Hard,8,A,20190920,105,104545,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,20.0,1805.0,6.0,4095.0,2019-9210-105
2802,2019-9210,Laver Cup,Hard,8,A,20190920,106,126774,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7.0,3420.0,30.0,1385.0,2019-9210-106
2803,2019-9210,Laver Cup,Hard,8,A,20190920,107,104745,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,9225.0,24.0,1450.0,2019-9210-107
2804,2019-9210,Laver Cup,Hard,8,A,20190920,108,106233,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,4575.0,33.0,1310.0,2019-9210-108


In [51]:
missing_ratio = players["dob"].isna().mean()
print(f"{missing_ratio:.2%} of dob values are missing.")

27.89% of dob values are missing.


We need to fill missing data for dob.
What we can do is calculate the dob by assuming a debut age of 18 (average debut age of tennis players) and using the date of their first professional match.

This is also not optimal since we do not know if we really have the debut game of a player in the data set or if his age was actually 18 when he played professionally the first time. However, it gives us a reasonable estimate.

In [59]:
import os
import glob
def read_all_pointbypoint_data(directory, file_pattern="*-points.csv"):
    """
    Reads all point-by-point data files from a directory and concatenates them into a single DataFrame.

    Args:
        directory (str): Directory containing point-by-point CSV files.

    Returns:
        pd.DataFrame: Concatenated DataFrame of all point-by-point data.
    """
    pattern = os.path.join(directory, file_pattern)
    file_list = glob.glob(pattern)

    if not file_list:
        raise FileNotFoundError(
            f"No files matching pattern '{pattern}' found in directory '{directory}'."
        )

    pointbypoint_data_list = [pd.read_csv(file) for file in file_list]
    for x in pointbypoint_data_list:
        print(f"Loaded {x.shape[0]} rows from {x.columns.tolist()}")
    combined_pointbypoint_data = pd.concat(pointbypoint_data_list, ignore_index=True)
    return combined_pointbypoint_data

points_data = read_all_pointbypoint_data('../data/slam_pointbypoint')
print(points_data.shape)
print(points_data.columns)
print(points_data.dtypes)
points_data = points_data[points_data['ElapsedTime'].notna()]
points_data

/var/folders/fm/xhk7242x0d30x60k6sqjyxl80000gn/T/ipykernel_54904/2918831955.py:21: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  pointbypoint_data_list = [pd.read_csv(file) for file in file_list]


Loaded 31851 rows from ['match_id', 'ElapsedTime', 'SetNo', 'P1GamesWon', 'P2GamesWon', 'SetWinner', 'GameNo', 'GameWinner', 'PointNumber', 'PointWinner', 'PointServer', 'Speed_KMH', 'Rally', 'P1Score', 'P2Score', 'P1Momentum', 'P2Momentum', 'P1PointsWon', 'P2PointsWon', 'P1Ace', 'P2Ace', 'P1Winner', 'P2Winner', 'P1DoubleFault', 'P2DoubleFault', 'P1UnfErr', 'P2UnfErr', 'P1NetPoint', 'P2NetPoint', 'P1NetPointWon', 'P2NetPointWon', 'P1BreakPoint', 'P2BreakPoint', 'P1BreakPointWon', 'P2BreakPointWon', 'P1FirstSrvIn', 'P2FirstSrvIn', 'P1FirstSrvWon', 'P2FirstSrvWon', 'P1SecondSrvIn', 'P2SecondSrvIn', 'P1SecondSrvWon', 'P2SecondSrvWon', 'P1ForcedError', 'P2ForcedError', 'History', 'Speed_MPH', 'P1BreakPointMissed', 'P2BreakPointMissed', 'ServeIndicator', 'Serve_Direction', 'Winner_FH', 'Winner_BH', 'ServingTo', 'P1TurningPoint', 'P2TurningPoint', 'ServeNumber', 'WinnerType', 'WinnerShotType', 'P1DistanceRun', 'P2DistanceRun', 'RallyCount', 'ServeWidth', 'ServeDepth', 'ReturnDepth']
Loaded 3

,match_id,ElapsedTime,SetNo,P1GamesWon,P2GamesWon,SetWinner,GameNo,GameWinner,PointNumber,PointWinner,...,P2TurningPoint,ServeNumber,WinnerType,WinnerShotType,P1DistanceRun,P2DistanceRun,RallyCount,ServeWidth,ServeDepth,ReturnDepth
0,2018-usopen-1101,0:00:00,1.0,0.0,0.0,0.0,1.0,0.0,0X,0.0,...,NaN,0.0,0,0,0.000,0.000,0.0,NaN,NaN,NaN
1,2018-usopen-1101,0:00:00,1.0,0.0,0.0,0.0,1.0,0.0,0Y,0.0,...,NaN,0.0,0,0,0.000,0.000,0.0,NaN,NaN,NaN
2,2018-usopen-1101,0:00:00,1.0,0.0,0.0,0.0,1.0,0.0,1,1.0,...,NaN,1.0,0,0,5.951,2.944,2.0,W,CTL,NaN
3,2018-usopen-1101,0:00:19,1.0,0.0,0.0,0.0,1.0,0.0,2,2.0,...,NaN,1.0,0,0,2.875,2.285,1.0,NaN,NaN,ND
4,2018-usopen-1101,0:00:37,1.0,0.0,0.0,0.0,1.0,0.0,3,2.0,...,NaN,2.0,0,0,3.886,3.986,1.0,BW,NCTL,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877719,2011-usopen-2701,1:10:11,2.0,3.0,5.0,0.0,9.0,0.0,98,2.0,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1877720,2011-usopen-2701,1:10:52,2.0,3.0,5.0,0.0,9.0,0.0,99,1.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1877721,2011-usopen-2701,1:11:50,2.0,3.0,5.0,0.0,9.0,0.0,100,1.0,...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1877722,2011-usopen-2701,1:12:23,2.0,3.0,5.0,0.0,9.0,0.0,101,2.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
points_data = points_data[['match_id', 'ElapsedTime', 'PointServer', 'Speed_KMH', 'ServeIndicator']]
points_data

,match_id,ElapsedTime,PointServer,Speed_KMH,ServeIndicator
0,2018-usopen-1101,0:00:00,0.0,0.0,0.0
1,2018-usopen-1101,0:00:00,0.0,0.0,0.0
2,2018-usopen-1101,0:00:00,2.0,178.0,2.0
3,2018-usopen-1101,0:00:19,2.0,180.0,2.0
4,2018-usopen-1101,0:00:37,2.0,152.0,2.0
...,...,...,...,...,...
1877719,2011-usopen-2701,1:10:11,1.0,167.0,1.0
1877720,2011-usopen-2701,1:10:52,1.0,183.0,1.0
1877721,2011-usopen-2701,1:11:50,1.0,168.0,1.0
1877722,2011-usopen-2701,1:12:23,1.0,143.0,1.0


In [62]:
import os
import glob


def read_all_pointbypoint_data(directory, file_pattern="*-matches.csv"):
    """
    Reads all point-by-point data files from a directory and concatenates them into a single DataFrame.

    Args:
        directory (str): Directory containing point-by-point CSV files.

    Returns:
        pd.DataFrame: Concatenated DataFrame of all point-by-point data.
    """
    pattern = os.path.join(directory, file_pattern)
    file_list = glob.glob(pattern)

    if not file_list:
        raise FileNotFoundError(
            f"No files matching pattern '{pattern}' found in directory '{directory}'."
        )

    pointbypoint_data_list = [pd.read_csv(file) for file in file_list]
    for x in pointbypoint_data_list:
        print(f"Loaded {x.shape[0]} rows from {x.columns.tolist()}")
    combined_pointbypoint_data = pd.concat(pointbypoint_data_list, ignore_index=True)
    return combined_pointbypoint_data

matches_data = read_all_pointbypoint_data('../data/slam_pointbypoint')
matches_data

Loaded 254 rows from ['match_id', 'year', 'slam', 'match_num', 'player1', 'player2', 'status', 'winner', 'event_name', 'round', 'court_name', 'court_id', 'player1id', 'player2id', 'nation1', 'nation2']
Loaded 253 rows from ['match_id', 'year', 'slam', 'match_num', 'player1', 'player2', 'status', 'winner', 'event_name', 'round', 'court_name', 'court_id', 'player1id', 'player2id', 'nation1', 'nation2']
Loaded 253 rows from ['match_id', 'year', 'slam', 'match_num', 'player1', 'player2', 'status', 'winner', 'event_name', 'round', 'court_name', 'court_id', 'player1id', 'player2id', 'nation1', 'nation2']
Loaded 187 rows from ['match_id', 'year', 'slam', 'match_num', 'player1', 'player2', 'status', 'winner', 'event_name', 'round', 'court_name', 'court_id', 'player1id', 'player2id', 'nation1', 'nation2']
Loaded 252 rows from ['match_id', 'year', 'slam', 'match_num', 'player1', 'player2', 'status', 'winner', 'event_name', 'round', 'court_name', 'court_id', 'player1id', 'player2id', 'nation1', '

,match_id,year,slam,match_num,player1,player2,status,winner,event_name,round,court_name,court_id,player1id,player2id,nation1,nation2
0,2019-ausopen-1101,2019,ausopen,1101,N. Djokovic,M. Krueger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SRB,USA
1,2019-ausopen-1102,2019,ausopen,1102,J. Tsonga,M. Klizan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRA,SVK
2,2019-ausopen-1103,2019,ausopen,1103,T. Daniel,T. Kokkinakis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JPN,AUS
3,2019-ausopen-1104,2019,ausopen,1104,P. Andujar,D. Shapovalov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ESP,CAN
4,2019-ausopen-1105,2019,ausopen,1105,D. Goffin,C. Garin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BEL,CHI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10508,2018-usopen-2503,2018,usopen,2503,Carla Suárez Navarro,Madison Keys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10509,2018-usopen-2504,2018,usopen,2504,Naomi Osaka,Lesia Tsurenko,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10510,2018-usopen-2601,2018,usopen,2601,Serena Williams,Anastasija Sevastova,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10511,2018-usopen-2602,2018,usopen,2602,Madison Keys,Naomi Osaka,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
import os
import glob
import pandas as pd
def read_all_atp_matches(directory, file_pattern="atp_matches_*.csv"):
    #
    # Reads all ATP match files from a directory and concatenates them into a single DataFrame.

    # Args:
    #     directory (str): Directory containing ATP match CSV files.

    # Returns:
    #     pd.DataFrame: Concatenated DataFrame of all ATP matches.
    #

    print(f"Reading ATP matches from directory: {directory}")
    pattern = os.path.join(directory, file_pattern)
    file_list = glob.glob(pattern)

    if not file_list:
        raise FileNotFoundError(
            f"No files matching pattern '{pattern}' found in directory '{directory}'."
        )

    atp_matches_list = [pd.read_csv(file) for file in file_list]
    combined_atp_matches = pd.concat(atp_matches_list, ignore_index=True)
    combined_atp_matches["match_id"] = (
        combined_atp_matches["tourney_id"]
        + "-"
        + combined_atp_matches["match_num"].astype("str")
    )
    print(
        f"Combined {len(combined_atp_matches)} ATP matches from {len(file_list)} files."
    )
    return combined_atp_matches

In [2]:
atp_matches = read_all_atp_matches('../data/atp_matches')
atp_matches

Reading ATP matches from directory: ../data/atp_matches


/var/folders/fm/xhk7242x0d30x60k6sqjyxl80000gn/T/ipykernel_14021/4247521459.py:24: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  atp_matches_list = [pd.read_csv(file) for file in file_list]


Combined 941649 ATP matches from 139 files.


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,match_id
0,2019-M020,Brisbane,Hard,32,A,20181231,300,105453,2.0,NaN,...,34.0,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0,2019-M020-300
1,2019-M020,Brisbane,Hard,32,A,20181231,299,106421,4.0,NaN,...,36.0,7.0,10.0,10.0,13.0,16.0,1977.0,239.0,200.0,2019-M020-299
2,2019-M020,Brisbane,Hard,32,A,20181231,298,105453,2.0,NaN,...,15.0,6.0,8.0,1.0,5.0,9.0,3590.0,40.0,1050.0,2019-M020-298
3,2019-M020,Brisbane,Hard,32,A,20181231,297,104542,NaN,PR,...,38.0,9.0,11.0,4.0,6.0,239.0,200.0,31.0,1298.0,2019-M020-297
4,2019-M020,Brisbane,Hard,32,A,20181231,296,106421,4.0,NaN,...,46.0,19.0,15.0,2.0,4.0,16.0,1977.0,18.0,1855.0,2019-M020-296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941644,2014-605,Tour Finals,Hard,8,F,20141109,514,104925,1.0,NaN,...,29.0,5.0,11.0,3.0,9.0,1.0,10010.0,5.0,4625.0,2014-605-514
941645,2014-605,Tour Finals,Hard,8,F,20141109,515,104925,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,10010.0,2.0,8700.0,2014-605-515
941646,2014-D015,Davis Cup WG F: FRA vs SUI,Clay,4,D,20141121,1,104527,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,5295.0,12.0,2740.0,2014-D015-1
941647,2014-D015,Davis Cup WG F: FRA vs SUI,Clay,4,D,20141121,2,104792,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,19.0,1825.0,2.0,9700.0,2014-D015-2


In [5]:
import pandas as pd

file_path = (
    "/Users/tim/Documents/Projects/TennisMatchAnalysis/data/test_data.parquet"
)

data = pd.read_parquet(file_path)

data

,tourney_id,tourney_name,draw_size,tourney_level,tourney_date,match_num,player1_id,player1_seed,player1_entry,player1_hand,...,player2_surface_Grass_avg,player2_surface_Carpet_avg,player2_surface_Unknown_avg,player2_years_on_tour,player2_pct_wimbledon_won,player2_pct_grand_slams_won,player2_pct_matches_won,player2_avg_rank_beaten,player2_avg_rank_points_beaten,player1_wins
0,1877-540,Wimbledon,32,G,1877-07-09,17,113999,0,DE,U,...,1.000000,0.0,0.0,0,1.000000,1.000000,0.000000,2367.000000,0.000000,1
1,1877-540,Wimbledon,32,G,1877-07-09,20,113987,0,DE,U,...,1.000000,0.0,0.0,0,1.000000,1.000000,1.000000,2367.000000,0.000000,1
2,1877-540,Wimbledon,32,G,1877-07-09,10,113969,0,DE,U,...,1.000000,0.0,0.0,0,0.000000,0.000000,0.500000,2367.000000,0.000000,0
3,1877-540,Wimbledon,32,G,1877-07-09,16,113976,0,DE,U,...,1.000000,0.0,0.0,0,0.500000,0.500000,0.000000,2367.000000,0.000000,0
4,1877-540,Wimbledon,32,G,1877-07-09,2,114146,0,DE,U,...,1.000000,0.0,0.0,0,0.333333,0.333333,0.333333,2367.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941644,2024-7696,Next Gen Finals,8,F,2024-12-18,386,210460,7,DE,R,...,0.037037,0.0,0.0,4,0.666667,0.650000,0.538745,392.404110,308.417808,0
941645,2024-7696,Next Gen Finals,8,F,2024-12-18,389,210506,2,DE,R,...,0.000000,0.0,0.0,2,0.666667,0.650000,0.696629,449.766667,197.568966,1
941646,2024-7696,Next Gen Finals,8,F,2024-12-18,398,210506,2,DE,R,...,0.000000,0.0,0.0,2,0.666667,0.650000,0.759124,807.951923,131.701923,0
941647,2024-7696,Next Gen Finals,8,F,2024-12-18,400,211663,8,DE,R,...,0.000000,0.0,0.0,2,0.666667,0.650000,0.760870,815.330097,142.304762,1
